In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from math import radians, cos, sin, asin, sqrt, atan2
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx

from collections import Counter
from datetime import datetime
from pyproj import Proj, transform

import rasterio
from rasterio.transform import xy
from rasterio.plot import show
from rasterio.features import geometry_mask

from shapely.geometry import Point, Polygon, LineString, MultiLineString

import statsmodels.formula.api as smf
import tqdm.notebook as tqdm

import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'retina'

In [6]:
data_path = '/home/gleb/Desktop/thesis/data/'

countries_path = data_path + 'world-administrative-boundaries/'
basins_6_path = data_path + 'HydroBASINS Africa 6 level/'
basins_12_path = data_path + 'HydroBASINS Africa 12 level/'
conflict_path = data_path + 'conflicts/'
dams_path = data_path + 'dams/'
sheds_file_3 = data_path + 'hyd_af_dem_3s/af_dem_3s.tif'
sheds_file_15 = data_path + 'hyd_af_dem_15s/hyd_af_dem_15s.tif'
rivers_file = data_path + 'HydroRIVERS_v10_af/HydroRIVERS_v10_af.gdb'

output_path = '/home/gleb/Desktop/thesis/outcomes/'

In [4]:
rivers = gpd.read_file(rivers_file)
levels = [1,2,3,4,5]
rivers_main = rivers[rivers['ORD_CLAS'].apply(lambda x: True if x in levels else False)].copy()
rivers_main.reset_index(inplace=True, drop=True)

In [5]:
with rasterio.open(sheds_file_15) as dem:
    dem_data = dem.read(1)
    transformation = dem.transform

In [4]:
def calculate_riv_gradient(elevation_data, transform, line_string):
    points = [xy for xy in line_string.coords]
    gradient_values = []
    distance_values = []
    for idx in range(len(points) - 1):
        point1, point2 = points[idx], points[idx + 1]
        col1, row1 = rasterio.transform.rowcol(transformation, point1[0], point1[1])
        col2, row2 = rasterio.transform.rowcol(transformation, point2[0], point2[1])
        elevation1 = dem_data[col1, row1]
#         print('Elevation level of point 1: ' + str(elevation1))
        elevation2 = dem_data[col2, row2]
#         print('Elevation level of point 2: ' + str(elevation2))
        distance = haversine(point1[0], point1[1], point2[0], point2[1])
#         print('Distance between points: ' + str(distance))
        gradient = ((elevation1 - elevation2) / distance)*100
#         print('Gradient between points: ' + str(gradient))
        distance_values.append(distance)
        gradient_values.append(gradient)
    return gradient_values, distance_values

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 * 1000
    return c * r

def gradient_fits(grad, category='suitable'):
    if category=='suitable':
        if (grad >= 1.5) & (grad <= 3):
            return True
        elif grad >= 6:
            return True
        else:
            return False
    elif category=='1.5-3':
        if (grad >= 1.5) & (grad <= 3):
            return True
        else:
            return False
    elif category == '3-6':
        if (grad > 3) & (grad < 6):
            return True
        else:
            return False
    elif category == '>6':
        if grad >= 6:
            return True 
        else:
            return False

# Test for river distances:

In [7]:
# if isinstance(river_geoms, MultiLineString):
#     grads, dists = calculate_gradient(dem_data, transformation, river_geoms[0])
index = 24740
river_geoms = rivers_main.loc[index, 'geometry']
grads, dists = calculate_riv_gradient(dem_data, transformation, river_geoms.geoms[0])
sum(dists)

7221.239512363846

In [8]:
rivers_main.iloc[index:index+1, :]

,HYRIV_ID,NEXT_DOWN,MAIN_RIV,LENGTH_KM,DIST_DN_KM,DIST_UP_KM,CATCH_SKM,UPLAND_SKM,ENDORHEIC,DIS_AV_CMS,ORD_STRA,ORD_CLAS,ORD_FLOW,HYBAS_L12,Shape_Length,geometry
24740,10025303,10024963,10018841,7.22,137.100006,11.1,25.48,25.5,0,0.047,1,2,8,1.120109e+09,0.07092,"MULTILINESTRING ((-6.55833 33.26250, -6.55208 ..."


# Basins:

In [9]:
bas_6 = gpd.read_file(basins_6_path + 'hybas_af_lev06_v1c.shp')
bas_12 = gpd.read_file(basins_12_path + 'hybas_af_lev12_v1c.shp')
bas_12['PFAF_ID_6l'] = bas_12['PFAF_ID'].apply(lambda x: int(str(x)[:6]))
bas_l12_to_6 = pd.Series(bas_12.PFAF_ID_6l.values,index=bas_12.HYBAS_ID).to_dict()

In [ ]:
# Calculate IV and geographic controls for each basin: 
# 1. Revir Gradient = ratio_of_grad, check
# 2. basin size = SUB_AREA (in sq.km), check
# 3. elevation = average_elevation, check
# 4. average basin gradient = average_gradient, check
# 5. river length= tot_riv_dist, check

In [10]:
# Matching rivers to basins of 6th level using PFAF ids:
rivers['HYBAS_L6_PFAFID'] = rivers['HYBAS_L12'].apply(lambda x: bas_l12_to_6[x])
rivers_main['HYBAS_L6_PFAFID'] = rivers_main['HYBAS_L12'].apply(lambda x: bas_l12_to_6[x])

# Dictionary of rivers geometries. Keys: rivers' HYRIV_ID. Values: geometries.
riv_geoms = pd.Series(rivers.geometry.values,index=rivers.HYRIV_ID).to_dict()

# Dictionary basins-rivers. Keys: Basin's PFAF index. Values: lists of 'HYRIV_ID' that belong to a basin.
bas_riv_dict = rivers_main.groupby('HYBAS_L6_PFAFID')['HYRIV_ID'].apply(list).to_dict()
# bas_riv_dict = rivers.groupby('HYBAS_L6_PFAFID')['HYRIV_ID'].apply(list).to_dict()

In [11]:
# Calculate ratios of suitable river gradients and total length of rivers for basins:
ratios_RG = {}
ratios_15_3 = {}
ratios_3_6 = {}
ratios_more_6 = {}
basin_riv_dist = {}

for pfaf_id in set(bas_6.PFAF_ID):
    print('Calculated ratio of IDs: ' + str(len(ratios_RG)/len(set(bas_6.PFAF_ID))), end = '\r')
    if pfaf_id in bas_riv_dict.keys():
        rivers_in_basin = bas_riv_dict[pfaf_id]
        
        total_rivers_distance = 0
        fitted_gradients_distance_RG = 0
        fitted_gradients_distance_15_3 = 0
        fitted_gradients_distance_3_6 = 0
        fitted_gradients_distance_more_6 = 0
        
        for riv in rivers_in_basin:
            river_geom = riv_geoms[riv]
            grads, dists = calculate_riv_gradient(dem_data, transformation, river_geom.geoms[0])
            # Which sections of the river have gradient 1.5-3% or >3%:
            grad_fits_RG = [gradient_fits(grad, category='suitable') for grad in grads]
            grad_fits_15_3 = [gradient_fits(grad, category='1.5-3') for grad in grads]
            grad_fits_3_6 = [gradient_fits(grad, category='3-6') for grad in grads]
            grad_fits_more_6 = [gradient_fits(grad, category='>6') for grad in grads]
            # Calculate distance of sections within needed gradient:
            fitted_gradients_distance_RG += sum([x for x, y in zip(dists, grad_fits_RG) if y])
            fitted_gradients_distance_15_3 += sum([x for x, y in zip(dists, grad_fits_15_3) if y])
            fitted_gradients_distance_3_6 += sum([x for x, y in zip(dists, grad_fits_3_6) if y])
            fitted_gradients_distance_more_6 += sum([x for x, y in zip(dists, grad_fits_more_6) if y])
            # Add length of the river to the distance of all rivers in the basin:
            total_rivers_distance += sum(dists)
        ratio_RG = fitted_gradients_distance_RG/total_rivers_distance
        ratio_15_3 = fitted_gradients_distance_15_3/total_rivers_distance
        ratio_3_6 = fitted_gradients_distance_3_6/total_rivers_distance
        ratio_more_6 = fitted_gradients_distance_more_6/total_rivers_distance
        
        ratios_RG[pfaf_id] = ratio_RG
        ratios_15_3[pfaf_id] = ratio_15_3
        ratios_3_6[pfaf_id] = ratio_3_6
        ratios_more_6[pfaf_id] = ratio_more_6
        basin_riv_dist[pfaf_id] = total_rivers_distance
    else:
        ratios_RG[pfaf_id] = 0
        ratios_15_3[pfaf_id] = 0
        ratios_3_6[pfaf_id] = 0
        ratios_more_6[pfaf_id] = 0
        basin_riv_dist[pfaf_id] = 0

In [12]:
# bas_6['RG'] = bas_6['grad_3_6'] 
bas_6['RG'] = bas_6['PFAF_ID'].apply(lambda x: ratios_RG[x])
bas_6['grad_15_3'] = bas_6['PFAF_ID'].apply(lambda x: ratios_15_3[x])
bas_6['grad_3_6'] = bas_6['PFAF_ID'].apply(lambda x: ratios_3_6[x])
bas_6['grad_more_6'] = bas_6['PFAF_ID'].apply(lambda x: ratios_more_6[x])
bas_6['tot_riv_dist'] = bas_6['PFAF_ID'].apply(lambda x: int(basin_riv_dist[x]))

In [5]:
# bas_6.to_file(output_path + 'hybas_af_lev06_v1c_grads.shp')
bas_6 = gpd.read_file(output_path + 'hybas_af_lev06_v1c_grads.shp')
bas_6.rename(columns={'grad_more_':'grad_more_6', 'tot_riv_di':'tot_riv_dist'}, inplace=True)

In [16]:
# Calculate average elevation and gradient for basins:
def calculate_bas_controls(geom, dem_file=dem_data):
    mask = geometry_mask([geom], out_shape=dem_file.shape, transform=transformation,
                         all_touched=False, invert=False)
    # Calculate average elevation:
    basin_elevations = np.ma.array(dem_data, mask=mask)
    av_elevation = basin_elevations.mean()
    basin_elevations = basin_elevations[~basin_elevations.mask].data
    n_pixels = len(basin_elevations)
    pxs_25 = len(basin_elevations[basin_elevations<=250])
    pxs_25_50 = len(basin_elevations[(basin_elevations>250) & (basin_elevations<=500)])
    pxs_50_1k = len(basin_elevations[(basin_elevations>500) & (basin_elevations<=1000)])
    pxs_more_1k = len(basin_elevations[(basin_elevations>1000) & (basin_elevations<=10000)])
    share_less_25 = pxs_25/n_pixels
    share_25_50 = pxs_25_50/n_pixels
    share_50_1k = pxs_50_1k/n_pixels
    share_more_1k = pxs_more_1k/n_pixels
    return [av_elevation, share_less_25, share_25_50, share_50_1k, share_more_1k]

bas_6[['av_elev', 'shr_l_25', 'shr_25_50', 'shr_50_1k', 'shr_m_1k']] = pd.DataFrame(bas_6['geometry'].apply(
    lambda x: calculate_bas_controls(x)).tolist(), index= bas_6.index)



In [5]:
# bas_6.to_file(output_path + '5rivers_controls.shp')
bas_6 = gpd.read_file(output_path + '5rivers_controls.shp')
bas_6.rename(columns={'grad_more_':'grad_more_6', 'tot_riv_di':'tot_riv_dist'}, inplace=True)
bas_6['HYBAS_ID'] = bas_6['HYBAS_ID'].apply(lambda x: str(x))
bas_6['PFAF_ID'] = bas_6['PFAF_ID'].apply(lambda x: str(x))

# Compose data on dams:

In [14]:
# Download dams and filter only dams for African continent
dams_df = gpd.read_file(dams_path + 'GRanD_Version_1_3/GRanD_dams_v1_3.shp')
countries = gpd.read_file(countries_path + 'world-administrative-boundaries.shp')
regions = ['Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa']
countries = countries[countries['region'].apply(lambda x: True if x in regions else False)]
countries_dict = {"CÃ´te d'Ivoire": 'Ivory Coast',
                  'Democratic Republic of the Congo': 'Congo (DRC)',
                  'Libyan Arab Jamahiriya': 'Libya',
                  'United Republic of Tanzania':'Tanzania'}
countries['name'] = countries['name'].apply(lambda x: countries_dict.get(x, x))
countries = set(countries['name'])

dams_df = dams_df[dams_df['COUNTRY'].apply(lambda x: True if x in countries else False)]
dams_df.reset_index(inplace=True,drop=True)

In [15]:
# import sys
# !{sys.executable} -m pip install openpyxl

# Add dams from FHReD 2015
fhred = pd.read_excel(dams_path + 'FHReD_2015_future_dams_Zarfl_et_al_beta_version.xlsx', sheet_name = 1)
fhred = fhred[(fhred['Continent'] == 'Africa')&(fhred['End'] <= 2023)].copy()
fhred['DAM_ID'] = fhred['DAM_ID'] + 10000
# Drop Continent, Capacity (MW), Stage, Start, Reference 1, Reference 2, Reference 3 columns
fhred.drop(columns=['Continent', 'Capacity (MW)', 'Stage', 'Start', 'Reference 1', 'Reference 2', 'Reference 3'], inplace=True)
cols_to_rename = {'DAM_ID':'GRAND_ID', 'Project name':'DAM_NAME', 'Country':'COUNTRY', 'Main_river':'RIVER', 'Major Basin':'MAIN_BASIN', 'Lon_Cleaned':'LONG_DD', 'LAT_cleaned':'LAT_DD', 'End':'YEAR'}
fhred.rename(columns = cols_to_rename, inplace=True)
fhred['USE_ELEC'] = 'Major'
fhred['USE_IRRI'] = 'Major'
fhred.reset_index(inplace=True, drop=True)

geometry = [Point(xy) for xy in zip(fhred['LONG_DD'], fhred['LAT_DD'])]
fhred_gpd = gpd.GeoDataFrame(fhred, geometry=geometry)
fhred_gpd.set_crs('epsg:4326', inplace=True)
print()

In [16]:
aqua = pd.read_excel(dams_path + 'Africa-dams_eng.xlsx', sheet_name = 1)
cols_to_rename = {'Reservoir capacity (million m3)': 'CAP_MAX', 'Dam height (m)': 'DAM_HGT_M', 'Alternate dam name':'DAM_NAME', 
                  'Country':'COUNTRY', 'River':'RIVER', 'Major basin':'MAIN_BASIN',
                  'Decimal degree longitude':'LONG_DD', 'Decimal degree latitude':'LAT_DD',
                  'Completed /operational since':'YEAR'}
aqua.rename(columns = cols_to_rename, inplace=True)
aqua['USE_IRRI'] = aqua['Irrigation '].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_ELEC'] = aqua['Hydroelectricity (MW)'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_SUPP'] = aqua['Water supply'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_FCON'] = aqua['Flood control'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_RECR'] = aqua['Recreation'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_NAVI'] = aqua['Navigation'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_PCON'] = aqua['Pollution control'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_LIVE'] = aqua['Livestock rearing'].apply(lambda x: 'Major' if x == 'x' else None)
aqua['USE_OTHR'] = aqua['Other'].apply(lambda x: 'Major' if x == 'x' else None)

cols_to_remove = ['Name of dam', 'ISO alpha- 3', 'Administrative\nUnit', 'Nearest city', 'Sub-basin', 'Reservoir area (km2)',
                  'Sedimen-tation \n(latest known) \n(%) ', 'National reference(s)', 'Other reference(s)', 'Comments', 'Irrigation ',
                  'Hydroelectricity (MW)','Water supply','Flood control','Recreation','Navigation','Pollution control','Livestock rearing','Other']
aqua.drop(columns= cols_to_remove, inplace=True)

aqua['YEAR'] = aqua['YEAR'].apply(lambda x: x if type(x) == int else None)
aqua = aqua[~aqua['YEAR'].isna()].copy()
aqua['YEAR'] = aqua['YEAR'].apply(lambda x: int(x))
aqua = aqua[~aqua['LAT_DD'].isna()].copy()
aqua.reset_index(inplace=True, drop=True)
aqua['GRAND_ID'] = range(20001, 20001 + len(aqua))

geometry = [Point(xy) for xy in zip(aqua['LONG_DD'], aqua['LAT_DD'])]
aqua_gpd = gpd.GeoDataFrame(aqua, geometry=geometry)
aqua_gpd.set_crs('epsg:4326', inplace=True)
print()

In [17]:
dams_df = pd.concat([dams_df, fhred_gpd, aqua_gpd], ignore_index=True)
dams_df = dams_df.sort_values(by='YEAR', ascending=False)
dams_df.drop_duplicates(subset=['geometry'], keep='first', inplace=True)

del aqua
del fhred

In [19]:
import time
import itertools
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 * 1000
    return c * r

# distance = haversine(point1[0], point1[1], point2[0], point2[1])

dams_df = dams_df.sort_values(by='GRAND_ID', ascending=False)

G = nx.Graph()
for index, row in dams_df.iterrows():
    # print(index)
    # print('Lon: ' + str(row['LONG_DD']) + ', Lat: ' + str(row['LAT_DD']))
    # print('X: ' + str(row.geometry.x) + ', Y: ' + str(row.geometry.x))
    G.add_node(row['GRAND_ID'], year=row['YEAR'], pos=(row.geometry.x, row.geometry.y))

for (id1, geom1, year1), (id2, geom2, year2) in itertools.combinations(dams_df[['GRAND_ID', 'geometry', 'YEAR']].values, 2):
    # Calculate the distance in meters
    # distance = haversine(lon1, lat1, lon2, lat2)
    distance = haversine(geom1.x, geom1.y, geom2.x, geom2.y)
    if distance > 1000:
        continue
    # Add an edge between the nodes with the distance as weight and years as attributes
    G.add_edge(id1, id2, weight=distance, year1=year1, year2=year2)

df = nx.to_pandas_edgelist(G, source='id1', target='id2')
dams_to_remove = set(df['id1'])
dams_df['duplicate'] = dams_df['GRAND_ID'].apply(lambda x: 1 if x in dams_to_remove else 0)
dams_df = dams_df[dams_df['duplicate'] == 0]
dams_df.reset_index(inplace=True, drop=True)
dams_df.to_file(output_path + 'combined_dams.shp')

## Assign basins to the countries and vice versa:

In [25]:
bas_temp = bas_6[['HYBAS_ID', 'geometry']].copy()
# Create centroids of basins:
bas_temp["geometry"] = bas_temp["geometry"].centroid
# Read boundaries of countries:
countries = gpd.read_file(countries_path + 'world-administrative-boundaries.shp')
cntry_name_match = {'Bosnia & Herzegovina':'Bosnia and Herzegovina',
                    'Democratic Republic of the Congo':'Congo (DRC)',
                    'Iran (Islamic Republic of)':'Iran',
                    "CÃ´te d'Ivoire":'Ivory Coast',"Lao People's Democratic Republic":'Laos',
                    'Libyan Arab Jamahiriya':'Libya', "Ma'tan al-Sarra":'Libya',
                    'The former Yugoslav Republic of Macedonia':'Macedonia',
                    'Moldova, Republic of':'Moldova', 'Myanmar':'Myanmar (Burma)',
                    "Democratic People's Republic of Korea": 'North Korea',
                    'Russian Federation':'Russia', 'Republic of Korea':'South Korea',
                    'Syrian Arab Republic':'Syria', 'United Republic of Tanzania':'Tanzania',
                    'U.K. of Great Britain and Northern Ireland':'United Kingdom','United States of America':'United States'}

countries['name'] = countries['name'].apply(lambda x: cntry_name_match[x] if x in cntry_name_match.keys() else x)
afr_countries = countries[countries.continent == 'Africa'].copy()
afr_countries.reset_index(inplace=True, drop=True)

# Match centroids of basins with countries:
joined = gpd.sjoin(bas_temp, afr_countries, how='left', op='within')
joined = joined[~joined[['index_right']].isna().any(axis=1)].reset_index(drop=True)
bas_country_dict = pd.Series(joined.name.values,index=joined.HYBAS_ID).to_dict()

# Create lists of basins for each corresponding country:
country_bas_dict = {}
for country, basins_in_country in joined.groupby('name'):
    bas_ids = basins_in_country['HYBAS_ID'].to_list()
    country_bas_dict[country] = bas_ids
    
# Hold accounts of not assigned basins
not_assigned_bas = set(joined[joined[['iso3']].isna().any(axis=1)]['HYBAS_ID'])

In [26]:
# # Manually add coastal basins for which centroid lies outside of the country borders:
# Extend not assigned basins dict:
not_assigned_bas = {'1060008110': 'Somalia',
                    # '1060003780':,
                    '1060020500': 'Gabon',
                    '1060023020': 'Nigeria',
                    '1060032860': 'Libyan Arab Jamahiriya',
                    # '1060034490': 'Sao Tome and Principe',
                    # '1060034610': ,
                    # '1060034900': ,
                    '1060035090': 'United Republic of Tanzania',
                    # '1060040030': 'Madagascar',
                    # '1060040040': 'Madagascar',
                    # '1060040050': 'Comoros',
                    # '1060040110': 'Seychelles',
                    # '1060040140': 'Seychelles',
                    # '1060040160': ,
                    # '1060040180':
                    }

# Apply dicts to fill the countries for basins:
bas_6['Country'] = bas_6['HYBAS_ID'].apply(lambda x: bas_country_dict.get(x, 'None'))
bas_6['Country'] = bas_6['Country'].apply(lambda x: not_assigned_bas.get(x, x))

# Drop basins w/o assigned countries (these basins have coastal and island nature): 
bas_6 = bas_6[bas_6['Country'] != 'None']
bas_6.reset_index(inplace=True,drop=True)

In [27]:
# Adjust the disputed territories:
country_adjust = {"Hala'ib Triangle": 'Egypt',
                  'Ilemi Triangle': 'Kenya'}
bas_6['Country'] = bas_6['Country'].apply(lambda x: country_adjust[x] if x in country_adjust.keys() else x)

# # Following the identification strategy, I remove countries which occupy only 1 hydrobasin
# Find countries with 1 hydrobasin:
countries_to_remove = set()
for country in country_bas_dict.keys():
    if len(country_bas_dict[country]) == 1:
        countries_to_remove.add(country)

# {'Burundi', 'Djibouti', 'Ilemi Triangle'}

bas_6 = bas_6[~bas_6['Country'].isin(countries_to_remove)].copy()
bas_6.reset_index(drop=True,inplace=True)

In [29]:
# bas_6.to_file(output_path + '5rivers_all_controls.shp')
bas_6 = gpd.read_file(output_path + '5rivers_all_controls.shp')
bas_6.rename(columns={'grad_more_':'grad_more_6', 'tot_riv_di':'tot_riv_dist'}, inplace=True)
bas_6['HYBAS_ID'] = bas_6['HYBAS_ID'].apply(lambda x: str(x))
bas_6['PFAF_ID'] = bas_6['PFAF_ID'].apply(lambda x: str(x))

In [30]:
# Explode the basin data frame by month (1-12) and year (1997-2023) to a panel dataset:
bas_6['year'] = bas_6['HYBAS_ID'].apply(lambda x: [year for year in range(1999, 2024)])
bas_6_exp = bas_6.explode('year', ignore_index=True)
# bas_6_exp['month'] = bas_6_exp['HYBAS_ID'].apply(lambda x: [month for month in range(1, 13)])
# bas_6_exp = bas_6_exp.explode('month', ignore_index=True)

## Match dams per country per year with basins:

In [95]:
# # IDs of dams that were removed in corresponding years:
# rem_dams = dams_df[dams_df['REM_YEAR'] != -99].copy().reset_index(drop=True)
# # rem_dams = pd.Series(rem_dams.REM_YEAR.values,index=rem_dams.GRAND_ID).to_dict()
# year_rem_dict = {}
# for year, data in rem_dams.groupby('REM_YEAR'):
#     dam_ids = data['GRAND_ID'].to_list()
#     year_rem_dict[year] = dam_ids
# year_rem_dict

{}

In [31]:
# # Calculate the total number of dams in country c in year t:
# Split the dams df:
year_to_use = 1999
# dams_df = gpd.read_file(dams_path + 'GRanD_dams_v1_3.shp')
dams_df = gpd.read_file(output_path + 'combined_dams.shp')

dams_before_start = dams_df[dams_df['YEAR'] < year_to_use]

# Create a dict of countries and dams:
dams_p_cntr_at_start = dams_before_start.groupby('COUNTRY').count().reset_index()
dams_p_cntr_at_start = pd.Series(dams_p_cntr_at_start.GRAND_ID.values,
                                index=dams_p_cntr_at_start.COUNTRY).to_dict()

# Create column in for dams per country in 1999:
bas_6_exp['dams_per_count_991'] = bas_6_exp['Country'].apply(lambda x: dams_p_cntr_at_start.get(x, 0))

In [32]:
# # Calculate dams for country c for year t:
# Split the df:
dams_after_start = dams_df[dams_df['YEAR'] >= year_to_use]
dams_after_start = dams_after_start.groupby(['COUNTRY', 'YEAR']).count().reset_index()

# Create the dict where key is tuple of country and year:
new_dams_after_start = dict()
for index, row in dams_after_start.iterrows():
    key = (row['COUNTRY'], row['YEAR'])
    new_dams_after_start[key] = row['GRAND_ID']

# Create the dict of total amount of dams in country c for year t:
counts = set(bas_6_exp.Country)
tot_dams_per_country_year = {}
idx = 0
for country in counts:
    idx += 1
    print(idx/len(counts), end='\r')
    tot_dams_per_year = {}
    for year in range(1999, 2024):
        if year == 1999:
            if country in dams_p_cntr_at_start.keys():
                tot_dams_per_year[year] = dams_p_cntr_at_start[country]
            else:
                tot_dams_per_year[year] = 0
        else:
            key = (country, year)
            if key in new_dams_after_start.keys():
                tot_dams_per_year[year] = tot_dams_per_year[year-1] + new_dams_after_start[key]
            else:
                tot_dams_per_year[year] = tot_dams_per_year[year-1]
    tot_dams_per_country_year[country] = tot_dams_per_year
    
# Use the created dictionary to fill in the basins dataframe:
bas_6_exp['n_of_dam_in_c_per_y'] = 0
for idx, row in bas_6_exp.iterrows():
    print(idx/len(bas_6_exp), end='\r')
    bas_6_exp.loc[idx, 'n_of_dam_in_c_per_y'] = tot_dams_per_country_year[row['Country']][row['year']]

In [34]:
# bas_6_exp.to_file(output_path + '5rivers_all_controls_panel.shp')
bas_6_exp = gpd.read_file(output_path + '5rivers_all_controls_panel.shp')
bas_6_exp.rename(columns={'grad_more_':'grad_more_6', 'tot_riv_di':'tot_riv_dist',
                      'n_of_dam_i':'dams_in_c_per_y', 'dams_per_c':'dams_per_c_99'}, inplace=True)
bas_6_exp['HYBAS_ID'] = bas_6_exp['HYBAS_ID'].apply(lambda x: str(x))
bas_6_exp['PFAF_ID'] = bas_6_exp['PFAF_ID'].apply(lambda x: str(x))
bas_6_exp['year'] = bas_6_exp['year'].apply(lambda x: int(x))
# bas_6_exp = bas_6_exp[bas_6_exp['year'] < 2018]
# bas_6_exp.reset_index(drop=True, inplace=True)

# Conflicts:

In [35]:
# CRS of df is EPSG:3857
df = gpd.read_file(conflict_path + '1997-01-01-2023-09-30.csv', sep=';', dtype={'timestamp': 'object'})
gdf_conf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitude, df.latitude))
gdf_conf.set_crs('epsg:4326', inplace=True)
del df
# gdf_conf.set_crs('epsg:3857', inplace=True)
# gdf_conf.to_crs(epsg=4326, inplace = True)

In [36]:
regions = ['Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa']
gdf_conf = gdf_conf[gdf_conf['region'].apply(lambda x: True if x in regions else False)]
gdf_conf['year'] = gdf_conf['year'].apply(lambda x: int(x))
# gdf_conf = gdf_conf[gdf_conf['year'] < 2018]
gdf_conf.reset_index(drop=True, inplace=True)

In [147]:
# s = gdf_conf[(gdf_conf['interaction'] == '18')][['actor1', 'actor2', 'notes']] – Beginning for cleaning of intrastate conflict

In [37]:
# All conflicts:
all_conflicts_gdf = gdf_conf.copy()

# Riots:
riots_gdf = gdf_conf[gdf_conf.event_type == 'Riots'].copy()
riots_gdf.reset_index(drop=True, inplace=True)

# Filter by battles:
battles_gdf = gdf_conf[gdf_conf.event_type == 'Battles'].copy()
battles_gdf.reset_index(drop=True, inplace=True)

del gdf_conf

In [38]:
# Match geography of battles with hydrobasins:
joined = gpd.sjoin(bas_6[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']],
                   all_conflicts_gdf[['event_id_cnty', 'event_date', 'geometry', 'year', 'fatalities', 'timestamp']], how='right')
joined.drop(['index_left'], axis=1, inplace=True)
joined[['day', 'month', 'year']] = joined['event_date'].str.split(' ', expand = True)
joined['month_year'] = joined[['month', 'year']].apply(lambda x: ' '.join(x), axis=1)
joined['month_year'] = joined['month_year'].apply(lambda x: datetime.strptime(x, '%B %Y'))
joined['month'] = joined['month_year'].apply(lambda x: x.month)
joined['year'] = joined['month_year'].apply(lambda x: x.year)

# Create dict of lists of conflicts for every combination with of basin ID, month and year:
basin_conf_dict = {}
# for hydrobasin_id, confs_in_basin in joined.groupby(['HYBAS_ID', 'month', 'year']):
for hydrobasin_id, confs_in_basin in joined.groupby(['HYBAS_ID', 'year']):
    confs_ids = confs_in_basin['event_id_cnty'].tolist()
    basin_conf_dict[hydrobasin_id] = confs_ids
    
# Match number of conflicts with basins, country and year:
bas_6_exp['conflicts_per_y'] = 0
for idx, row in bas_6_exp.iterrows():
    print(idx/len(bas_6_exp), end='\r')
    # key = (row['HYBAS_ID'], row['month'], row['year'])
    key = (row['HYBAS_ID'], row['year'])
    if key in basin_conf_dict.keys():
        bas_6_exp.loc[idx, 'conflicts_per_y'] = int(len(basin_conf_dict[key]))
    else:
        bas_6_exp.loc[idx, 'conflicts_per_y'] = int(0)

In [39]:
# Match geography of battles with hydrobasins:
joined = gpd.sjoin(bas_6[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']],
                   riots_gdf[['event_id_cnty', 'event_date', 'geometry', 'year', 'fatalities', 'timestamp']], how='right')
joined.drop(['index_left'], axis=1, inplace=True)
joined[['day', 'month', 'year']] = joined['event_date'].str.split(' ', expand = True)
joined['month_year'] = joined[['month', 'year']].apply(lambda x: ' '.join(x), axis=1)
joined['month_year'] = joined['month_year'].apply(lambda x: datetime.strptime(x, '%B %Y'))
joined['month'] = joined['month_year'].apply(lambda x: x.month)
joined['year'] = joined['month_year'].apply(lambda x: x.year)

# Create dict of lists of conflicts for every combination with of basin ID, month and year:
basin_conf_dict = {}
# for hydrobasin_id, confs_in_basin in joined.groupby(['HYBAS_ID', 'month', 'year']):
for hydrobasin_id, confs_in_basin in joined.groupby(['HYBAS_ID', 'year']):
    confs_ids = confs_in_basin['event_id_cnty'].tolist()
    basin_conf_dict[hydrobasin_id] = confs_ids
    
# Match number of conflicts with basins, country and year:
bas_6_exp['riots_per_y'] = 0
for idx, row in bas_6_exp.iterrows():
    print(idx/len(bas_6_exp), end='\r')
    # key = (row['HYBAS_ID'], row['month'], row['year'])
    key = (row['HYBAS_ID'], row['year'])
    if key in basin_conf_dict.keys():
        bas_6_exp.loc[idx, 'riots_per_y'] = int(len(basin_conf_dict[key]))
    else:
        bas_6_exp.loc[idx, 'riots_per_y'] = int(0)

In [40]:
# Match geography of battles with hydrobasins:
joined = gpd.sjoin(bas_6[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']],
                   battles_gdf[['event_id_cnty', 'event_date', 'geometry', 'year', 'fatalities', 'timestamp']], how='right')
joined.drop(['index_left'], axis=1, inplace=True)
joined[['day', 'month', 'year']] = joined['event_date'].str.split(' ', expand = True)
joined['month_year'] = joined[['month', 'year']].apply(lambda x: ' '.join(x), axis=1)
joined['month_year'] = joined['month_year'].apply(lambda x: datetime.strptime(x, '%B %Y'))
joined['month'] = joined['month_year'].apply(lambda x: x.month)
joined['year'] = joined['month_year'].apply(lambda x: x.year)

# Create dict of lists of conflicts for every combination with of basin ID, month and year:
basin_conf_dict = {}
# for hydrobasin_id, confs_in_basin in joined.groupby(['HYBAS_ID', 'month', 'year']):
for hydrobasin_id, confs_in_basin in joined.groupby(['HYBAS_ID', 'year']):
    confs_ids = confs_in_basin['event_id_cnty'].tolist()
    basin_conf_dict[hydrobasin_id] = confs_ids
    
# Match number of conflicts with basins, country and year:
bas_6_exp['battles_per_y'] = 0
for idx, row in bas_6_exp.iterrows():
    print(idx/len(bas_6_exp), end='\r')
    # key = (row['HYBAS_ID'], row['month'], row['year'])
    key = (row['HYBAS_ID'], row['year'])
    if key in basin_conf_dict.keys():
        bas_6_exp.loc[idx, 'battles_per_y'] = int(len(basin_conf_dict[key]))
    else:
        bas_6_exp.loc[idx, 'battles_per_y'] = int(0)

In [41]:
bas_6_exp.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'RG', 'grad_15_3', 'grad_3_6', 'grad_more_6', 'tot_riv_dist',
       'av_elev', 'shr_l_25', 'shr_25_50', 'shr_50_1k', 'shr_m_1k', 'has_dam',
       'n_of_dams', 'Country', 'year', 'dams_per_c_99', 'dams_in_c_per_y',
       'geometry', 'conflicts_per_y', 'riots_per_y', 'battles_per_y'],
      dtype='object')

In [42]:
# bas_6_exp.battles_per_m_loop.value_counts()
bas_6_exp['had_conf'] = bas_6_exp['conflicts_per_y'].apply(lambda x: 1 if x > 0 else 0)
bas_6_exp['had_riot'] = bas_6_exp['riots_per_y'].apply(lambda x: 1 if x > 0 else 0)
bas_6_exp['had_fight'] = bas_6_exp['battles_per_y'].apply(lambda x: 1 if x > 0 else 0)
bas_6_exp['RGxD_hat'] = bas_6_exp['RG']*bas_6_exp['dams_in_c_per_y']

In [44]:
# bas_6_exp.to_file(output_path + '5rivers_prepared_data.shp')
bas_6_exp = gpd.read_file(output_path + '5rivers_prepared_data.shp')
bas_6_exp.rename(columns={'grad_more_':'grad_more_6', 'tot_riv_di':'tot_riv_dist',
                      'dams_in_c_':'dams_in_c_per_y', 'dams_per_c':'dams_per_c_99',
                      'conflicts_': 'confs_py', 'riots_per_':'riots_py', 
                      'battles_pe':'battles_py'}, inplace=True)
# In case of weird column: dams_per_1 = dams_per_c_99

## Add dams as endogenous variable:

In [45]:
year_to_use = 1999
# dams_df = gpd.read_file(dams_path + 'GRanD_dams_v1_3.shp')
dams_df = gpd.read_file(output_path + 'combined_dams.shp')

# # Wherever main year is -99, so is alternative year:
dams_df = dams_df[dams_df['YEAR'] != -99]
dams_df.reset_index(inplace=True,drop=True)

uses = {'Main', 'Major'}
# uses = {'Main', 'Sec', 'Major'}

dams_df['el_ir'] = dams_df.apply(lambda row: 1 if (row.USE_ELEC in uses) or 
                                 (row.USE_IRRI in uses) else 0, axis=1)
dams_df = dams_df[dams_df['el_ir'] == 1]
dams_df.reset_index(inplace=True, drop=True)

dams_1900_1999 = dams_df[(dams_df['YEAR'] >= 1900) & (dams_df['YEAR'] < year_to_use)].copy()
dams_1900_1999.reset_index(inplace=True, drop=True)

dams_after_99 = dams_df[dams_df['YEAR'] >= year_to_use].copy()
dams_after_99.reset_index(inplace=True, drop=True)

In [46]:
# Get the amount of dams per basin by 1999:
joined_1900_1999 = gpd.sjoin(dams_1900_1999[['GRAND_ID', 'YEAR', 'MAIN_USE', 'geometry']],
                   bas_6[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']], op='within', how='left')
joined_1900_1999 = joined_1900_1999[~joined_1900_1999['HYBAS_ID'].isna()]
joined_1900_1999.reset_index(inplace=True, drop=True)

dam_bas_1900_99 = {}
for bas_id, data in joined_1900_1999.groupby('HYBAS_ID'):
    dam_ids = data['GRAND_ID'].to_list()
    dam_bas_1900_99[bas_id] = dam_ids

# Fill number of dams by 1999 for each basin with created dictionary values:
bas_6_exp['basdams_99'] = bas_6_exp['HYBAS_ID'].apply(lambda x: len(dam_bas_1900_99[x]) if 
                                                      x in dam_bas_1900_99.keys() else 0)

In [47]:
# Get the amount of dams per basin per year after 1999:
joined_after_99 = gpd.sjoin(dams_after_99[['GRAND_ID', 'YEAR', 'MAIN_USE', 'geometry']],
                   bas_6[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']], op='within', how='left')
joined_after_99 = joined_after_99[~joined_after_99['HYBAS_ID'].isna()]
joined_after_99.reset_index(inplace=True, drop=True)

dam_bas_after_99 = {}
for bas_id, data in joined_after_99.groupby(['HYBAS_ID', 'YEAR']):
    dam_ids = data['GRAND_ID'].to_list()
    dam_bas_after_99[bas_id] = dam_ids

In [48]:
# removed_dams = {1987: [1918],
#                 2002: [2085],
#                 2003: [769],
#                 2005: [2006],
#                 2007: [820, 2844],
#                 2008: [772, 2882],
#                 2013: [6315],
#                 2016: [6285]}

# dams_df[dams_df.REM_YEAR != -99].COUNTRY.value_counts()

# dam_bas_after_99
# dam_bas_1900_99

In [49]:
# Create the dict of total amount of dams in basin c for year t:
basins = set(bas_6_exp.HYBAS_ID)
tot_dams_per_basin_year = {}
idx = 0
for basin in basins:
    idx += 1
    print(idx/len(basins), end='\r')
    tot_dams_per_year = {}
    for year in range(1999, 2024):
        if year == 1999:
            if basin in dam_bas_1900_99.keys():
                tot_dams_per_year[year] = len(dam_bas_1900_99[basin])
            else:
                tot_dams_per_year[year] = 0
        else:
            key = (basin, year)
            if key in dam_bas_after_99.keys():
                tot_dams_per_year[year] = tot_dams_per_year[year-1] + len(dam_bas_after_99[key])
            else:
                tot_dams_per_year[year] = tot_dams_per_year[year-1]
    tot_dams_per_basin_year[basin] = tot_dams_per_year
    
# Use the created dictionary to fill in the basins dataframe:
bas_6_exp['dams_in_b_per_y'] = 0
for idx, row in bas_6_exp.iterrows():
    print(idx/len(bas_6_exp), end='\r')
    bas_6_exp.loc[idx, 'dams_in_b_per_y'] = tot_dams_per_basin_year[row['HYBAS_ID']][row['year']]

In [50]:
# bas_6_exp.drop(columns=['basdams_py'], inplace=True)

col_names = list(bas_6_exp.columns)
for col in col_names:
    if len(col) > 10:
        print(col)

grad_more_6
tot_riv_dist
dams_per_c_99
dams_in_c_per_y
dams_in_b_per_y


In [51]:
cols_to_rename = {'grad_more_6': 'grad_m_6',
                  'tot_riv_dist': 'tot_riv_di',
                  'dams_per_c_99': 'dams_pc_99',
                  'dams_in_b_per_y': 'dams_pby',
                  'n_of_dam_i': 'dams_pcy',
                  'dams_in_c_per_y': 'dams_pcy',}

bas_6_exp.rename(columns=cols_to_rename, inplace=True)

In [176]:
bas_6_exp.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'RG', 'grad_15_3', 'grad_3_6', 'grad_m_6', 'tot_riv_di',
       'av_elev', 'shr_l_25', 'shr_25_50', 'shr_50_1k', 'shr_m_1k', 'has_dam',
       'n_of_dams', 'Country', 'year', 'dams_pc_99', 'n_of_dam_i', 'confs_py',
       'riots_py', 'battles_py', 'had_conf', 'had_riot', 'had_fight',
       'RGxD_hat', 'geometry', 'basdams_99', 'dams_pby'],
      dtype='object')

In [52]:
# Interaction between geographic controls and D_hat
bas_6_exp['z_ieygg'] = bas_6_exp['grad_15_3']*bas_6_exp['dams_pcy']
bas_6_exp['z_tkdgw'] = bas_6_exp['grad_3_6']*bas_6_exp['dams_pcy']
bas_6_exp['z_sobhm'] = bas_6_exp['grad_m_6']*bas_6_exp['dams_pcy']
bas_6_exp['z_fobki'] = bas_6_exp['tot_riv_di']*bas_6_exp['dams_pcy']
bas_6_exp['z_hyeah'] = bas_6_exp['av_elev']*bas_6_exp['dams_pcy']
bas_6_exp['z_oxjpe'] = bas_6_exp['SUB_AREA']*bas_6_exp['dams_pcy']
bas_6_exp['z_vcjei'] = bas_6_exp['shr_l_25']*bas_6_exp['dams_pcy']
bas_6_exp['z_nlvsk'] = bas_6_exp['shr_25_50']*bas_6_exp['dams_pcy']
bas_6_exp['z_ahjvn'] = bas_6_exp['shr_50_1k']*bas_6_exp['dams_pcy']
bas_6_exp['z_zgjij'] = bas_6_exp['shr_m_1k']*bas_6_exp['dams_pcy']

columns_names_meanings = {
    'z_ieygg': 'Share of river gradient between 1,5% and 3% in a basin multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_tkdgw': 'Share of river gradient between 3% and 6% in a basin multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_sobhm': 'Share of river gradient more than 6% in a basin multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_fobki': 'Total distance of rivers in a basin multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_hyeah': 'Average elevation of a basin multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_vcjei': 'Area of elevation lower than 250m as a share of basin area, multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_nlvsk': 'Area of elevation between 250m and 500m as a share of basin area, multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_ahjvn': 'Area of elevation between 500m and 1000m as a share of basin area, multiplied by total number of dams in a corresponding country in a corresponding year.',
    'z_zgjij': 'Area of elevation between 500m and 1000m as a share of basin area, multiplied by total number of dams in a corresponding country in a corresponding year.'
    }

In [53]:
bas_6_exp["centroid"] = bas_6_exp["geometry"].centroid
bas_6_exp["lon"] = bas_6_exp["centroid"].apply(lambda p: p.x)
bas_6_exp["lat"] = bas_6_exp["centroid"].apply(lambda p: p.y)
bas_6_exp.drop(columns=['centroid'], inplace= True)
bas_6_exp.to_file(output_path + '5rivers_fully_prepared_data.shp')

# Advanced First Stage Check:

In [64]:
# bas_6_exp = gpd.read_file(output_path + '5rivers_fully_prepared_data.shp')
bas_6_exp.rename(columns={'grad_more_':'grad_more_6', 'tot_riv_di':'tot_riv_dist',
                      'dams_in_c_':'dams_in_c_per_y', 'dams_per_c':'dams_per_c_99',
                      'conflicts_': 'confs_py', 'riots_per_':'riots_py', 
                      'battles_pe':'battles_py'}, inplace=True)

iv = bas_6_exp[bas_6_exp['year'] == 1999].copy()
iv.reset_index(drop=True,inplace=True)
iv['has_dam'] = iv['basdams_99'].apply(lambda x: 1 if x > 0 else 0)

In [56]:
dams_df = gpd.read_file(output_path + 'combined_dams.shp')

any_dams_99 = dams_df[dams_df['YEAR'] <= 1999].copy()
any_dams_99.reset_index(inplace=True,drop=True)

# uses = {'Major'}
uses = {'Main', 'Major'}
# uses = {'Main', 'Sec', 'Major'}

any_dams_99['el_ir'] = any_dams_99.apply(lambda row: 1 if (row.USE_ELEC in uses) or 
                                              (row.USE_IRRI in uses) else 0, axis=1)
any_dams_99['el'] = any_dams_99.apply(lambda row: 1 if row.USE_ELEC in uses else 0, axis=1)
any_dams_99['ir'] = any_dams_99.apply(lambda row: 1 if row.USE_IRRI in uses else 0, axis=1)

dams_el_ir_99 = any_dams_99[any_dams_99['el_ir'] == 1]
dams_el_ir_99.reset_index(inplace=True,drop=True)
dams_el_99 = any_dams_99[any_dams_99['el'] == 1]
dams_el_99.reset_index(inplace=True,drop=True)
dams_ir_99 = any_dams_99[any_dams_99['ir'] == 1]
dams_ir_99.reset_index(inplace=True,drop=True)

In [59]:
# Hydropower dams before 1999:
joined = gpd.sjoin(dams_el_99[['GRAND_ID', 'YEAR', 'MAIN_USE', 'geometry']],
                   iv[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']], op='within', how='left')
basin_dam_dict = {}
for hydrobasin_id, dams_in_basin in joined.groupby('HYBAS_ID'):
    dam_ids = dams_in_basin['GRAND_ID'].tolist()
    basin_dam_dict[hydrobasin_id] = dam_ids    
iv['has_dam'] = iv['HYBAS_ID'].apply(lambda x: 1 if x in basin_dam_dict.keys() else 0)
df1 = pd.DataFrame(iv.drop(columns='geometry'))

first_stage1 = smf.ols("has_dam ~ SUB_AREA + av_elev + shr_25_50 + shr_50_1k + shr_m_1k + tot_riv_dist + grad_15_3 + grad_3_6 + grad_m_6",
                      data=df1).fit()

print(first_stage1.summary())
rows, model = [], first_stage1
for idx in range (len(first_stage1.params)-3, len(first_stage1.params)):
    rows.append([model.params.index[idx], model.params[idx], model.pvalues[idx]])
print(pd.DataFrame(rows, columns=[' ', 'Coef:', 'P>|t|']))

                            OLS Regression Results                            
Dep. Variable:                has_dam   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     14.10
Date:                Fri, 05 Jan 2024   Prob (F-statistic):           1.38e-22
Time:                        16:42:50   Log-Likelihood:                 2010.6
No. Observations:                3579   AIC:                            -4001.
Df Residuals:                    3569   BIC:                            -3939.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0051      0.007      0.776   

In [62]:
# All dams before 1999:
joined = gpd.sjoin(any_dams_99[['GRAND_ID', 'YEAR', 'MAIN_USE', 'geometry']],
                   iv[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']], op='within', how='left')
basin_dam_dict = {}
for hydrobasin_id, dams_in_basin in joined.groupby('HYBAS_ID'):
    dam_ids = dams_in_basin['GRAND_ID'].tolist()
    basin_dam_dict[hydrobasin_id] = dam_ids    
iv['has_dam'] = iv['HYBAS_ID'].apply(lambda x: 1 if x in basin_dam_dict.keys() else 0)
df1 = pd.DataFrame(iv.drop(columns='geometry'))

first_stage1 = smf.ols("has_dam ~ SUB_AREA + av_elev + shr_25_50 + shr_50_1k + shr_m_1k + tot_riv_dist + grad_15_3 + grad_3_6 + grad_m_6",
                      data=df1).fit()

print(first_stage1.summary())
rows, model = [], first_stage1
for idx in range (len(first_stage1.params)-3, len(first_stage1.params)):
    rows.append([model.params.index[idx], model.params[idx], model.pvalues[idx]])
print(pd.DataFrame(rows, columns=[' ', 'Coef:', 'P>|t|']))

                            OLS Regression Results                            
Dep. Variable:                has_dam   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     25.87
Date:                Fri, 05 Jan 2024   Prob (F-statistic):           1.29e-43
Time:                        17:18:39   Log-Likelihood:                -691.82
No. Observations:                3579   AIC:                             1404.
Df Residuals:                    3569   BIC:                             1465.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0407      0.014      2.942   

In [63]:
# All dams before 1999:
joined = gpd.sjoin(dams_el_99[['GRAND_ID', 'YEAR', 'MAIN_USE', 'geometry']],
                   iv[['HYBAS_ID', 'PFAF_ID', 'geometry', 'RG']], op='within', how='left')
basin_dam_dict = {}
for hydrobasin_id, dams_in_basin in joined.groupby('HYBAS_ID'):
    dam_ids = dams_in_basin['GRAND_ID'].tolist()
    basin_dam_dict[hydrobasin_id] = dam_ids
iv['has_dam'] = iv['HYBAS_ID'].apply(lambda x: 1 if x in basin_dam_dict.keys() else 0)
df1 = pd.DataFrame(iv.drop(columns='geometry'))

first_stage1 = smf.ols("has_dam ~ SUB_AREA + av_elev + shr_25_50 + shr_50_1k + shr_m_1k + tot_riv_dist + RG",
                      data=df1).fit()

print(first_stage1.summary())
rows, model = [], first_stage1
for idx in range (len(first_stage1.params)-3, len(first_stage1.params)):
    rows.append([model.params.index[idx], model.params[idx], model.pvalues[idx]])
print(pd.DataFrame(rows, columns=[' ', 'Coef:', 'P>|t|']))

                            OLS Regression Results                            
Dep. Variable:                has_dam   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     17.81
Date:                Fri, 05 Jan 2024   Prob (F-statistic):           2.18e-23
Time:                        17:21:47   Log-Likelihood:                 2009.5
No. Observations:                3579   AIC:                            -4003.
Df Residuals:                    3571   BIC:                            -3954.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0050      0.006      0.791   